# Train Toxicity Model

This notebook trains a model to detect toxicity in online comments. It uses a CNN architecture for text classification trained on the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973) and pre-trained GloVe embeddings which can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/).

This model is a modification of [example code](https://github.com/fchollet/keras/blob/master/examples/pretrained_word_embeddings.py) found in the [Keras Github repository](https://github.com/fchollet/keras) and released under an [MIT license](https://github.com/fchollet/keras/blob/master/LICENSE). For further details of this license, find it [online](https://github.com/fchollet/keras/blob/master/LICENSE) or in this repository in the file KERAS_LICENSE. 

## Usage Instructions
(TODO: nthain) - Move to README

Prior to running the notebook, you must:

* Download the [Wikipedia Talk Labels: Toxicity dataset](https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973)
* Download pre-trained [GloVe embeddings](http://nlp.stanford.edu/data/glove.6B.zip)
* (optional) To skip the training step, you will need to download a model and tokenizer file. We are looking into the appropriate means for distributing these (sometimes large) files.

In [ ]:
!pip install --upgrade tensorflow==1.4

    100% |████████████████████████████████| 40.8MB 2.8kB/s ta 0:00:011


In [7]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd

from model_tool import ToxModel
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
import re



ImportError: cannot import name np_utils

In [6]:
!pip install keras==2.0.6

    100% |████████████████████████████████| 235kB 541kB/s ta 0:00:01
    100% |████████████████████████████████| 2.8MB 224kB/s ta 0:00:01
  Running setup.py bdist_wheel for keras ... done
  Stored in directory: /home/dcek/.cache/pip/wheels/c2/80/ba/2beab8c2131e2dcc391ee8a2f55e648af66348115c245e0839
  Running setup.py bdist_wheel for theano ... done
  Stored in directory: /home/dcek/.cache/pip/wheels/46/a2/7d/b4cac381d5151daa9f9e0b3e4e4b65edaea6355ae296c97cf2
Successfully built keras theano
  Found existing installation: Keras 2.1.3
    Uninstalling Keras-2.1.3:
      Successfully uninstalled Keras-2.1.3


## Load Data

In [2]:
training_3rd_vis = pd.read_csv('cleaned_final_train.csv')
display(training_3rd_vis["comment_text"].head(n=20))
n_records_features_vis_3rd = len(training_3rd_vis)
print(" Number of features {}".format(n_records_features_vis_3rd))
train = 'cleaned_final_train.csv'


0     explanation edits made username hardcore metal...
1     aww matches background colour seemingly stuck ...
2     hey man really trying edit war guy constantly ...
3     make real suggestions improvement wondered sec...
4                         sir hero chance remember page
5              congratulations well use tools well talk
6                           cocksucker piss around work
7     vandalism matt shirvington article reverted pl...
8     sorry word nonsense offensive anyway intending...
9                  alignment subject contrary dulithgow
10    fair use rationale image wonju jpg thanks uplo...
11                     bbq man lets discuss maybe phone
12    hey talk exclusive group wp talibans good dest...
13    start throwing accusations warnings lets revie...
14    oh girl started arguments stuck nose belong be...
15    juelz santanas age two zero zero two juelz san...
16              bye look come think comming back tosser
17      redirect talk voydan pop georgiev cherno

 Number of features 638284


In [3]:
model_list = []

## Train Models

In [ ]:
MODEL_NAME = 'augmentori_cleaned_singlebigru_fasttext_1st'
debias_random_model = ToxModel()
debias_random_model.train(2,train, text_column = 'comment_text', toxic = 'toxic', severe_toxic = 'severe_toxic', obscene = 'obscene', threat = 'threat', insult = 'insult', identity_hate = 'identity_hate', model_name = MODEL_NAME, model_list = model_list)

Hyperparameters
---------------
max_num_words: 199814
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 0.0007
es_patience: 1
batch_size: 256
embedding_dim: 300
epochs: 1
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: False
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
train_text_temp shape (638284, 250) and train_labels_temp shape (638284, 6)
 ---- 
train_text shape (638284, 250) and train_labels shape (638284, 6)
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 574456 samples, validate on 63828 samples
Epoch 1/1

Epoch 00001: val_loss improved from inf to 0.04554, saving model to models/augmentori_cleaned_singlebigru_fasttext_1st0_model.h5
 - 652s - loss: 0.0676 - acc: 0.9767 - val_loss: 0.0455 - val_acc: 0.9821
Epoch 0 auc 0.983094783184 best_auc -1
Train on 574456 samples, validate on 63828 sa

In [ ]:
from keras.models import load_model
import os
model_list = []
for fold_id in range(0, 10):
    model_path = 'augmentori_gru_lstm' + str(fold_id)
    model = load_model(
        os.path.join('models', '%s_model.h5' % model_path))
    model_list.append(model)
    

In [8]:
from keras.models import load_model
import numpy as np
import os
model_list = []
for fold_id in range(0, 10):
    model_path = 'augmentori_gru_lstm' + str(fold_id)
    model = load_model(
        os.path.join('models', '%s_model.h5' % model_path))
    model_path = os.path.join('models', "model{0}_weights.npy".format(fold_id))
    weights = np.load(model_path)
    model.set_weights(weights)
    model_list.append(model)

In [11]:
from keras.preprocessing.sequence import pad_sequences
import cPickle
import os
def prep_text(texts):
    """Turns text into into padded sequences.

    The tokenizer must be initialized before calling this method.

    Args:
      texts: Sequence of text strings.

    Returns:
      A tokenized and padded text sequence as a model input.
    """
    model_name = 'augmentori_gru_lstm'
    tokenizer = cPickle.load(
        open(
            os.path.join('models', '%s_tokenizer.pkl' % model_name),
            'rb'))
    text_sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(
        text_sequences, maxlen=250)

In [12]:

total_meta = []
meta_train = pd.read_csv('final_train.csv')
X_test = meta_train['comment_text']
X_test = prep_text(X_test)
X= X_test
fold_size = len(X) // 10
for fold_id in range(0, 10):
    fold_start = fold_size * fold_id
    fold_end = fold_start + fold_size
            
    if fold_id == 10 - 1:
        fold_end = len(X)

    train_x = np.concatenate([X[:fold_start], X[fold_end:]])

    val_x = X[fold_start:fold_end]
          
    meta = model_list[fold_id].predict(val_x, batch_size=128)
    if (fold_id == 0):
        total_meta = meta
    else:
        total_meta = np.concatenate((total_meta, meta), axis=0)

In [13]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
subm = pd.read_csv('sample_submission.csv')
submid = pd.DataFrame({'id': subm["id"]})
total_meta_data = pd.concat([submid, pd.DataFrame(total_meta, columns = label_cols)], axis=1)


In [14]:
display(total_meta_data.head(n=20))


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,5.298672e-07,1.361637e-10,7.528020e-09,3.818864e-10,3.872117e-08,7.897755e-10
1,0000247867823ef7,2.175462e-06,2.507667e-09,3.737669e-08,7.730432e-10,1.419462e-07,1.429161e-09
2,00013b17ad220c46,1.382709e-02,1.843092e-07,4.588822e-06,1.099868e-08,9.896796e-06,1.191894e-07
3,00017563c3f7919a,4.313957e-07,9.578123e-11,6.308026e-09,3.524564e-10,3.479985e-08,8.696782e-10
4,00017695ad8997eb,2.839943e-06,5.312022e-09,1.006481e-07,1.515298e-09,3.593553e-07,2.260464e-09
5,0001ea8717f6de06,2.226181e-06,1.201818e-08,6.024056e-08,2.033103e-09,2.641394e-07,3.800403e-09
6,00024115d4cbde0f,9.997441e-01,9.550502e-04,9.863858e-01,2.469440e-08,9.533083e-01,2.931226e-05
7,000247e83dcc1211,7.835670e-05,3.863176e-08,1.403170e-07,7.525878e-10,6.863291e-07,3.154716e-09
8,00025358d4737918,6.994727e-07,4.960894e-10,1.851791e-08,6.080487e-10,4.313723e-08,7.851165e-10
9,00026d1092fe71cc,1.341810e-06,1.451595e-09,2.378028e-08,4.642549e-10,8.334749e-08,4.743154e-10


In [15]:
total_meta_data.to_csv('augmentori_meta_grulstmCV_nopretrain.csv', index=False)

In [8]:
test_predicts = pd.read_csv('gru_cv_output.csv')
display(test_predicts.head(n=20))
test_predicts.shape


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999538,6.181733e-01,0.997640,5.096922e-02,9.898657e-01,3.255197e-01
1,0000247867823ef7,0.000013,1.247423e-07,0.000003,4.708379e-08,2.553956e-06,1.125692e-06
2,00013b17ad220c46,0.000055,1.836647e-06,0.000020,6.441840e-07,1.196271e-05,7.204705e-06
3,00017563c3f7919a,0.000013,2.737614e-07,0.000003,1.778666e-07,1.700491e-06,5.387048e-07
4,00017695ad8997eb,0.000919,9.784006e-06,0.000113,2.973163e-06,2.222102e-05,6.399844e-06
5,0001ea8717f6de06,0.000131,1.483309e-06,0.000048,9.474676e-07,1.413156e-05,4.073511e-06
6,00024115d4cbde0f,0.000043,6.720049e-07,0.000012,7.150590e-07,9.158607e-06,1.265853e-06
7,000247e83dcc1211,0.295549,2.245386e-04,0.005013,3.404184e-06,6.566032e-03,1.335455e-04
8,00025358d4737918,0.029627,5.836995e-06,0.000559,3.917941e-07,1.921604e-03,1.296143e-05
9,00026d1092fe71cc,0.000014,1.825587e-07,0.000002,4.109103e-07,1.300620e-06,3.998649e-07


(153164, 7)

### Random model

In [3]:
MODEL_NAME = 'multi-labelNLP_charrnn'
debias_random_model = ToxModel()
debias_random_model.train(1,train, text_column = 'comment_text', toxic = 'toxic', severe_toxic = 'severe_toxic', obscene = 'obscene', threat = 'threat', insult = 'insult', identity_hate = 'identity_hate', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 50000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 7e-05
es_patience: 1
batch_size: 128
embedding_dim: 300
epochs: 50
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
train_text_temp shape (159571, 250) and train_labels_temp shape (159571, 4)
 ---- 
valid_text shape (15958, 250) and valid_labels shape (15958, 4)
train_text shape (143613, 250) and train_labels shape (143613, 4)
Data prepared!
Loading embeddings...
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
nicknamer not in vocabulary
 not in vocabulary
 not in vocabulary
songfulness not in vocabulary
Elonore not in vocabulary
homager not in vocabulary
forebodingness not in vocabulary
 not in vocabulary
Fredek not in vocabulary
 not in v

 not in vocabulary
[] not in vocabulary List is empty
 not in vocabulary
 not in vocabulary
dissipatedness not in vocabulary
 not in vocabulary
 not in vocabulary
musicianships not in vocabulary
mahjongs not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
campanologist not in vocabulary
 not in vocabulary
reprinter not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
Christianizes not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
dyslexically not in vocabulary
preluder not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
moderatenesses not in vocabulary
Ingamar not in vocabulary
jiujitsus not in vocabulary
subcomputation not in vocabulary
 not in vocabulary
archfiends not in vocabulary
 not in vocabulary
foxtrotting not in vocabulary
electroencephalographs not in vocabulary
adversene

 not in vocabulary
Nikaniki not in vocabulary
Arianist not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
knowledgeableness not in vocabulary
[] not in vocabulary List is empty
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
zigzagger not in vocabulary
Exchequers not in vocabulary
 not in vocabulary
 not in vocabulary
magnetohydrodynamical not in vocabulary
[] not in vocabulary List is empty
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
Jurua not in vocabulary
 not in vocabulary
chanciness not in vocabulary
 not in vocabulary
maydays not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
trinketed not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
outargued not in vocabulary
Reichstags not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in voca

outfaced not in vocabulary
dumdums not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
Sanskritize not in vocabulary
 not in vocabulary
geocentricism not in vocabulary
 not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
uraniums not in vocabulary
 not in vocabulary
disreputableness not in vocabulary
monarchistic not in vocabulary
hdqrs not in vocabulary
Fahrenheits not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
disinformations not in vocabulary
pericardia not in vocabulary
 not in vocabulary
Nefen not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
Aguistin not in vocabulary
 not in vocabulary
Gayel not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabular

disreputes not in vocabulary
 not in vocabulary
Gerhardine not in vocabulary
 not in vocabulary
 not in vocabulary
monkeyshine not in vocabulary
hydraulicking not in vocabulary
friedcake not in vocabulary
dratting not in vocabulary
 not in vocabulary
Melamie not in vocabulary
 not in vocabulary
 not in vocabulary
womenfolks not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
doters not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
legmen not in vocabulary
Americanizations not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
clamorer not in vocabulary
 not in vocabulary
 not in vocabulary
Rayshell not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
promenader not in vocabulary
balefuller not in vocabulary
nonbelligerent not in vocabulary
 not in vocabulary
barning not

...

exacter not in vocabulary
 not in vocabulary
caviled not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
Heindrick not in vocabulary
 not in vocabulary
 not in vocabulary
Welshwoman not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
briner not in vocabulary
 not in vocabulary
 not in vocabulary
noninterchangeable not in vocabulary
[] not in vocabulary List is empty
 not in vocabulary
Valenka not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
camaraderies not in vocabulary
 not in vocabulary
Beitris not in vocabulary
vigilantist not in vocabulary
 not in vocabulary
Chiarra not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
greenswards not in vocabulary
 not in vocabulary
burnables not in vocabulary
flecker not in vocabulary
speedboating not in vocabulary
netts not in vocabulary
 not in vocabulary
 no

[] not in vocabulary List is empty
 not in vocabulary
 not in vocabulary
 not in vocabulary
bedaub not in vocabulary
mongolisms not in vocabulary
Aguistin not in vocabulary
 not in vocabulary
metropolitanization not in vocabulary
salacity not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
underenumerated not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
passioning not in vocabulary
[] not in vocabulary List is empty
colonelcies not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
bemire not in vocabulary
consumerisms not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
tandooris not in vocabulary
krills not in vocabulary
 not in vocabulary
polychemicals not in vocabulary
 not in vocabulary
foxtrotting not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
Gwenore not in vocabulary
 no

 not in vocabulary
 not in vocabulary
sigher not in vocabulary
chrisms not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
vealer not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
molybdenums not in vocabulary
 not in vocabulary
 not in vocabulary
argots not in vocabulary
 not in vocabulary
bleater not in vocabulary
separatenesses not in vocabulary
imping not in vocabulary
 not in vocabulary
 not in vocabulary
[] not in vocabulary List is empty
 not in vocabulary
 not in vocabulary
 not in vocabulary
seizors not in vocabulary
 not in vocabulary
brontosaurs not in vocabulary
[] not in vocabulary List is empty
ethnologies not in vocabulary
 not in vocabulary
misguidedness not in vocabulary
[] not in vocabulary List is empty
 not in vocabulary
kriegspiel not in vocabulary
 not in vocabulary
 not in vocabulary
 not in vocabulary
stinkingly not in vocabulary
 not in vocabulary
beriberis not in vocabulary
sinisterness not in vocabulary
connotative

Train on 143613 samples, validate on 15958 samples
Epoch 1/50
Epoch 00000: val_loss improved from inf to 0.08809, saving model to models/multi-labelNLP_model.h5
1049s - loss: 0.2646 - acc: 0.9442 - val_loss: 0.0881 - val_acc: 0.9708
Epoch 2/50
Epoch 00001: val_loss improved from 0.08809 to 0.06706, saving model to models/multi-labelNLP_model.h5
1045s - loss: 0.0788 - acc: 0.9715 - val_loss: 0.0671 - val_acc: 0.9708
Epoch 3/50
Epoch 00002: val_loss improved from 0.06706 to 0.05972, saving model to models/multi-labelNLP_model.h5
1011s - loss: 0.0646 - acc: 0.9774 - val_loss: 0.0597 - val_acc: 0.9833
Epoch 4/50
Epoch 00003: val_loss improved from 0.05972 to 0.05452, saving model to models/multi-labelNLP_model.h5
1000s - loss: 0.0581 - acc: 0.9799 - val_loss: 0.0545 - val_acc: 0.9837
Epoch 5/50
Epoch 00004: val_loss improved from 0.05452 to 0.05107, saving model to models/multi-labelNLP_model.h5
991s - loss: 0.0541 - acc: 0.9820 - val_loss: 0.0511 - val_acc: 0.9845
Epoch 6/50
Epoch 00005: 

ValueError: Unknown layer: Attention

In [8]:
MODEL_NAME = 'multi-labelNLP-second'
second_model = ToxModel()
second_model.train(0,train, text_column = 'comment_text', toxic = 'toxic', severe_toxic = 'severe_toxic', obscene = 'obscene', threat = 'threat', insult = 'insult', identity_hate = 'identity_hate', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 7e-05
es_patience: 1
batch_size: 128
embedding_dim: 300
epochs: 50
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: False
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
train_text_temp shape (159571, 250) and train_labels_temp shape (159571, 2)
 ---- 
valid_text shape (15958, 250) and valid_labels shape (15958, 2)
train_text shape (143613, 250) and train_labels shape (143613, 2)
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 143613 samples, validate on 15958 samples
Epoch 1/50
Epoch 00000: val_loss improved from inf to 0.19217, saving model to models/multi-labelNLP-second_model.h5
493s - loss: 0.3494 - acc: 0.9083 - val_loss: 0.1922 - val_acc: 0.9470
Epoch 2/50
Epoch 00001: val_loss improved from 

In [7]:
debias_random_model = ToxModel(model_name="multi-labelNLP-gru-cv0") 

Hyperparameters
---------------
max_num_words: 30000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 7e-05
embedding_dim: 300
cnn_kernel_sizes: [5, 5, 5]
es_patience: 1
epochs: 50
cnn_filter_sizes: [128, 128, 128]
batch_size: 128
model_name: multi-labelNLP-gru-cv
max_sequence_length: 250
stop_early: False
embedding_trainable: False



In [4]:
second_model = ToxModel(model_name="multi-labelNLP-second") 

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 7e-05
embedding_dim: 300
cnn_kernel_sizes: [5, 5, 5]
es_patience: 1
epochs: 50
cnn_filter_sizes: [128, 128, 128]
batch_size: 128
model_name: multi-labelNLP-second
max_sequence_length: 250
stop_early: False
embedding_trainable: False



In [8]:
import numpy as np
random_test = pd.read_csv('test.csv')
np.where(pd.isnull(random_test)) #check null rows

(array([], dtype=int64), array([], dtype=int64))

In [7]:
print(random_test.iloc[52300]) #print value of null row

id                                               56cf8b7315e85f14
comment_text    SOmebody fucked up the homepage plz edit!! tha...
Name: 52300, dtype: object


In [9]:
random_test = pd.read_csv('test.csv')
#random_test = random_test.dropna()
prediction = debias_random_model.predict(random_test['comment_text'])

In [10]:
prediction.shape

(153164, 6)

In [9]:
random_test = pd.read_csv('test.csv')
random_test = random_test.dropna()
print(random_test.iloc[52300])

id                                                   231302702569
comment_text    Just a note about external links: I have remov...
Name: 52301, dtype: object


In [10]:
random_test.shape

(226997, 2)

In [8]:
for id, p in enumerate(prediction):
    if(id <20):
        print(p)

[0.94351786 0.07261173 0.87614065 0.2488063 ]
[2.8036063e-04 2.0171970e-05 3.3794012e-04 2.7047885e-05]
[0.0013453  0.00012583 0.00167132 0.00017606]
[1.0656077e-04 9.6954127e-06 1.1833789e-04 1.3261302e-05]
[9.6572866e-04 7.3803014e-05 1.2766599e-03 1.0159020e-04]
[1.4914108e-04 1.2232612e-05 1.7201710e-04 1.6546424e-05]
[3.7022735e-04 3.1974312e-05 4.6733892e-04 4.4136799e-05]
[0.03845833 0.00216745 0.05712204 0.00358418]
[0.00292935 0.00019908 0.005152   0.00027639]
[1.1728035e-04 1.0837189e-05 1.2751747e-04 1.4890081e-05]
[0.06014245 0.00027489 0.07595153 0.00041313]
[0.0037365  0.00061411 0.00671029 0.00094831]
[1.1792146e-04 6.9626149e-06 1.3104505e-04 8.9941350e-06]
[1.12934598e-04 1.16250676e-05 1.20583965e-04 1.62092383e-05]
[9.21920000e-05 9.38676385e-06 1.02012295e-04 1.30086419e-05]
[3.8325472e-04 3.0754032e-05 4.9539335e-04 4.2015748e-05]
[0.00463269 0.00045546 0.00638342 0.000654  ]
[0.00668138 0.00048621 0.01024681 0.00069852]
[1.05928797e-04 1.01692503e-05 1.12349095e-0

In [9]:
#second model
random_test = pd.read_csv('test.csv')
random_test = random_test.dropna()
prediction_second = second_model.predict(random_test['comment_text'])
prediction_second.shape

(153164, 2)

In [10]:
for id, p in enumerate(prediction_second):
    if(id <20):
        print(p)

[0.98729837 0.32326353]
[8.3768519e-04 1.1632358e-06]
[1.1443039e-03 2.0584848e-06]
[4.5535603e-04 3.8094805e-07]
[2.3912357e-02 7.7905795e-05]
[5.241920e-04 4.858725e-07]
[2.1548306e-03 5.0328690e-06]
[0.73191476 0.0132686 ]
[0.03209163 0.00011017]
[4.660586e-04 4.495613e-07]
[0.43936914 0.00131104]
[0.30907333 0.00103553]
[6.220829e-04 5.968530e-07]
[5.3532032e-04 4.9457026e-07]
[4.8731244e-04 3.8716783e-07]
[1.6188634e-03 2.8966961e-06]
[1.9687539e-02 3.7851005e-05]
[1.3295321e-02 4.3206492e-05]
[5.089241e-04 4.337683e-07]
[1.7802878e-03 3.2394330e-06]


In [11]:
random_test = pd.read_csv('test.csv')
#random_test = random_test.dropna()
test_id = random_test['id'].astype(str)

In [12]:
test_id.shape

(153164,)

In [13]:
header = ["id"]
df = pd.DataFrame(test_id, columns=header)
#display(df.head(n=20))

df.id = df.id.astype("str")
print(df.dtypes)
display(df.head(n=20))
#print(np.where(pd.isnull(df)))
#print(df.shape)
#df.reset_index(drop=True, inplace=True)



id    object
dtype: object


,id
0,00001cee341fdb12
1,0000247867823ef7
2,00013b17ad220c46
3,00017563c3f7919a
4,00017695ad8997eb
5,0001ea8717f6de06
6,00024115d4cbde0f
7,000247e83dcc1211
8,00025358d4737918
9,00026d1092fe71cc


In [14]:
#IF NO SPLIT
headers = ["toxic","severe_toxic","obscene","threat","insult","identity_hate"]
test_df = pd.DataFrame(prediction, columns=headers, dtype=float)
display(test_df.head(n=20))
print(np.where(pd.isnull(test_df)))
print(test_df.shape)
test_df.reset_index(drop=True, inplace=True)


,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.999144,4.910571e-01,0.997201,1.553519e-02,9.842854e-01,1.343174e-01
1,0.000013,6.671647e-07,0.000010,1.191349e-07,4.216913e-06,5.983958e-07
2,0.000209,6.238616e-06,0.000085,1.348426e-06,3.153176e-05,5.135477e-06
3,0.000025,9.297542e-07,0.000012,3.098087e-07,4.252371e-06,1.555411e-07
4,0.002148,1.040108e-05,0.000469,3.862768e-05,1.052680e-04,2.039362e-05
5,0.000042,1.268990e-06,0.000016,2.848018e-07,4.899765e-06,3.813471e-07
6,0.000051,1.395919e-06,0.000016,6.946671e-07,7.570949e-06,4.054838e-07
7,0.071056,1.163827e-04,0.001971,1.682328e-06,8.182763e-04,6.897181e-06
8,0.801900,3.431676e-05,0.024083,2.839076e-06,1.149196e-01,1.346543e-05
9,0.000011,3.321367e-07,0.000007,5.973085e-08,1.487395e-06,1.684030e-07


(array([], dtype=int64), array([], dtype=int64))
(153164, 6)


In [14]:
#IF SPLIT
headers = ["toxic","severe_toxic"]
test_df_second = pd.DataFrame(prediction_second, columns=headers, dtype=float)
display(test_df_second.head(n=20))
print(np.where(pd.isnull(test_df_second)))
print(test_df_second.shape)
test_df_second.reset_index(drop=True, inplace=True)


,toxic,severe_toxic
0,0.987298,3.232635e-01
1,0.000838,1.163236e-06
2,0.001144,2.058485e-06
3,0.000455,3.809481e-07
4,0.023912,7.790580e-05
5,0.000524,4.858725e-07
6,0.002155,5.032869e-06
7,0.731915,1.326860e-02
8,0.032092,1.101697e-04
9,0.000466,4.495613e-07


(array([], dtype=int64), array([], dtype=int64))
(153164, 2)


In [15]:
headers = ["obscene","threat","insult","identity_hate"]
test_df = pd.DataFrame(prediction, columns=headers, dtype=float)
display(test_df.head(n=20))
print(np.where(pd.isnull(test_df)))
print(test_df.shape)
test_df.reset_index(drop=True, inplace=True)

,obscene,threat,insult,identity_hate
0,0.943518,0.072612,0.876141,0.248806
1,0.000280,0.000020,0.000338,0.000027
2,0.001345,0.000126,0.001671,0.000176
3,0.000107,0.000010,0.000118,0.000013
4,0.000966,0.000074,0.001277,0.000102
5,0.000149,0.000012,0.000172,0.000017
6,0.000370,0.000032,0.000467,0.000044
7,0.038458,0.002167,0.057122,0.003584
8,0.002929,0.000199,0.005152,0.000276
9,0.000117,0.000011,0.000128,0.000015


(array([], dtype=int64), array([], dtype=int64))
(153164, 4)


In [15]:
#IF NO SPLIT
df_new = pd.concat([df,test_df], axis=1)
#df_new = df.merge(test_df, how='outer')
#df_new.id = df_new.id.astype("int")
display(df_new.head(n=20))
print(df_new.dtypes)

np.where(pd.isnull(df_new))

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999144,4.910571e-01,0.997201,1.553519e-02,9.842854e-01,1.343174e-01
1,0000247867823ef7,0.000013,6.671647e-07,0.000010,1.191349e-07,4.216913e-06,5.983958e-07
2,00013b17ad220c46,0.000209,6.238616e-06,0.000085,1.348426e-06,3.153176e-05,5.135477e-06
3,00017563c3f7919a,0.000025,9.297542e-07,0.000012,3.098087e-07,4.252371e-06,1.555411e-07
4,00017695ad8997eb,0.002148,1.040108e-05,0.000469,3.862768e-05,1.052680e-04,2.039362e-05
5,0001ea8717f6de06,0.000042,1.268990e-06,0.000016,2.848018e-07,4.899765e-06,3.813471e-07
6,00024115d4cbde0f,0.000051,1.395919e-06,0.000016,6.946671e-07,7.570949e-06,4.054838e-07
7,000247e83dcc1211,0.071056,1.163827e-04,0.001971,1.682328e-06,8.182763e-04,6.897181e-06
8,00025358d4737918,0.801900,3.431676e-05,0.024083,2.839076e-06,1.149196e-01,1.346543e-05
9,00026d1092fe71cc,0.000011,3.321367e-07,0.000007,5.973085e-08,1.487395e-06,1.684030e-07


id                object
toxic            float64
severe_toxic     float64
obscene          float64
threat           float64
insult           float64
identity_hate    float64
dtype: object


(array([], dtype=int64), array([], dtype=int64))

In [18]:
#IF SPLIT
df_new = pd.concat([df,test_df_second,test_df], axis=1)
#df_new = df.merge(test_df, how='outer')
#df_new.id = df_new.id.astype("int")
display(df_new.head(n=20))
print(df_new.dtypes)

np.where(pd.isnull(df_new))

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.987298,3.232635e-01,0.943518,0.072612,0.876141,0.248806
1,0000247867823ef7,0.000838,1.163236e-06,0.000280,0.000020,0.000338,0.000027
2,00013b17ad220c46,0.001144,2.058485e-06,0.001345,0.000126,0.001671,0.000176
3,00017563c3f7919a,0.000455,3.809481e-07,0.000107,0.000010,0.000118,0.000013
4,00017695ad8997eb,0.023912,7.790580e-05,0.000966,0.000074,0.001277,0.000102
5,0001ea8717f6de06,0.000524,4.858725e-07,0.000149,0.000012,0.000172,0.000017
6,00024115d4cbde0f,0.002155,5.032869e-06,0.000370,0.000032,0.000467,0.000044
7,000247e83dcc1211,0.731915,1.326860e-02,0.038458,0.002167,0.057122,0.003584
8,00025358d4737918,0.032092,1.101697e-04,0.002929,0.000199,0.005152,0.000276
9,00026d1092fe71cc,0.000466,4.495613e-07,0.000117,0.000011,0.000128,0.000015


id                object
toxic            float64
severe_toxic     float64
obscene          float64
threat           float64
insult           float64
identity_hate    float64
dtype: object


(array([], dtype=int64), array([], dtype=int64))

In [18]:
#hds = ["id","toxic","severe_toxic","obscene","threat","insult","identity_hate"]

#df2 = pd.DataFrame([[231298963278,0.5,0.5,0.5,0.5,0.5,0.5]], columns = hds)
#df_newer = df_new.append(df2)

In [13]:
df_new.shape

(153164, 7)

In [16]:
head = ["id","toxic","severe_toxic","obscene","threat","insult","identity_hate"]
df_new.to_csv('cv_gru_output.csv', columns = head, index=False)

In [ ]:
for id, p in enumerate(prediction):
    

### Plain wikipedia model

In [7]:
MODEL_NAME = 'cnn_wiki_tox_v3'
wiki_model = ToxModel()
wiki_model.train(wiki['train'], wiki['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 95692 samples, validate on 32128 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.17471, saving model to ../models/cnn_wiki_tox_v3_model.h5
134s - loss: 0.2437 - acc: 0.9141 - val_loss: 0.1747 - val_acc: 0.9359
Epoch 2/20
Epoch 00001: val_loss improved from 0.17471 to 0.14997, saving model to ../models/cnn_wiki_tox_v3_model.h5
134s - loss: 0.1654 - acc: 0.9388 - val_loss: 0.1500 - val_acc: 0.9439
Epoch 3/20
Epoch 00002: val_loss improved from 0.14997 to 0.13735, saving mo

In [8]:
wiki_test = pd.read_csv(wiki['test'])
wiki_model.score_auc(wiki_test['comment'], wiki_test['is_toxic'])

0.95997760130597887

### Debiased model

In [9]:
MODEL_NAME = 'cnn_debias_tox_v3'
debias_model = ToxModel()
debias_model.train(debias['train'], debias['dev'], text_column = 'comment', label_column = 'is_toxic', model_name = MODEL_NAME)

Hyperparameters
---------------
max_num_words: 10000
dropout_rate: 0.3
verbose: True
cnn_pooling_sizes: [5, 5, 40]
es_min_delta: 0
learning_rate: 5e-05
es_patience: 1
batch_size: 128
embedding_dim: 100
epochs: 20
cnn_filter_sizes: [128, 128, 128]
cnn_kernel_sizes: [5, 5, 5]
max_sequence_length: 250
stop_early: True
embedding_trainable: False

Fitting tokenizer...
Tokenizer fitted!
Preparing data...
Data prepared!
Loading embeddings...
Embeddings loaded!
Building model graph...
Training model...
Train on 99157 samples, validate on 33283 samples
Epoch 1/20
Epoch 00000: val_loss improved from inf to 0.16575, saving model to ../models/cnn_debias_tox_v3_model.h5
140s - loss: 0.2258 - acc: 0.9215 - val_loss: 0.1657 - val_acc: 0.9406
Epoch 2/20
Epoch 00001: val_loss improved from 0.16575 to 0.14430, saving model to ../models/cnn_debias_tox_v3_model.h5
139s - loss: 0.1595 - acc: 0.9420 - val_loss: 0.1443 - val_acc: 0.9472
Epoch 3/20
Epoch 00002: val_loss improved from 0.14430 to 0.13724, savin

In [11]:
debias_test = pd.read_csv(debias['test'])
debias_model.prep_data_and_score(debias_test['comment'], debias_test['is_toxic'])

AttributeError: ToxModel instance has no attribute 'prep_data_and_score'